<a href="https://www.kaggle.com/code/houmannorasteh/fork-of-t5-fine-tuning?scriptVersionId=166816339" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [36]:
! pip install transformers
! pip install torch

### importing libraries 

In [37]:
import torch
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
from itertools import zip_longest

### defining general variables

In [38]:
#defining global valriables throughout the whole notebook
EPOCH = 10
BATCH_SIZE = 16
MAX_INPUT_LENGTH = 65
MAX_LABEL_LENGTH = 8
MODEL_LINK = "google/flan-t5-small"

In [39]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_LINK, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(MODEL_LINK).to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### reading dataset and cleaning it

In [40]:
# a funciton to read data off of a database link is here to help getting and organizing data into dataframes
def get_data(address):
    lines = []
    with open(address) as file:
        for line in file:
            x = json.loads(line)
            lines.append(x)
    sentences, orl, sep_sentences = [], [], []
    for i in range(len(lines)):
        sep_sentences.append(lines[i]['sentences'])
        sentences.append(' '.join(lines[i]['sentences']))
        orl.append(lines[i]['orl'])
    dataframe = pd.DataFrame({'sentence': sentences, 'orl': orl, 'sep_sent': sep_sentences})
    return dataframe

# this function is to make a list of the said attribute for later iterations
def list_of(attributes, requested_atr):
    requested_list = []
    for sublist in attributes:
        if sublist[-1] == requested_atr:
            requested_list.append(sublist)
    return requested_list

# this function was made to find target(s)/agent(s) of a dse according to list of attributes
def organize_data(attributes, sentence):
    AGENT, DSE, TARGET = '', '', ''
    target_flag, agent_flag = False, False
    for sublist in attributes:
        if sublist[-1] == 'DSE':
            dse_start = int(sublist[0])
            dse_end = int(sublist[1] + 1)
            DSE += ' '.join(sentence[dse_start:dse_end]) + '|'
            
            
            # looking for the targets and agents of this dse that we have found
            target_flag = False
            for sub_sublist in list_of(attributes, 'TARGET'):
                if sub_sublist[0] == dse_start and int(sub_sublist[1] + 1) == dse_end:
                    target_start = int(sub_sublist[2])
                    target_end = int(sub_sublist[3] + 1)
                    TARGET += ' '.join(sentence[target_start:target_end]) + '|'
                    target_flag = True
            if not target_flag:
                TARGET += ' |'
            
            agent_flag = False
            for sub_sublist in list_of(attributes, 'AGENT'):
                if sub_sublist[0] == dse_start and int(sub_sublist[1] + 1) == dse_end:
                    agent_start = int(sub_sublist[2])
                    agent_end = int(sub_sublist[3] + 1)
                    AGENT += ' '.join(sentence[agent_start:agent_end]) + '|'
                    agent_flag = True
            if not agent_flag:
                AGENT += ' |'
    return AGENT, DSE, TARGET

### getting training/validation/test data into data frames and dividing each of {agent, target, dse} elements

In [41]:
df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.train0.conll.json")

# Organize tarin data into diffrent columns
# for each dse, function will dicide if it has target(s)/agent(s) and divides them with <" | ">
for i in range(len(df)):
    agent, dse, target = organize_data(df['orl'][i], df['sep_sent'][i])
    df.loc[i, 'agent'] = agent
    df.loc[i, 'dse'] = dse
    df.loc[i, 'target'] = target

df.drop(columns='sep_sent')

,sentence,orl,agent,dse,target
0,The Kimberley Provincial Hospital said it woul...,"[[6, 8, 0, 3, AGENT], [6, 8, 6, 8, DSE], [6, 8...",The Kimberley Provincial Hospital|,would probably know|,whether one of its patients had Congo Fever|
1,Saeed said indications were that those tests w...,"[[1, 1, 0, 0, AGENT], [1, 1, 1, 1, DSE], [1, 1...",Saeed|,said|,those tests|
2,He said it was his opinion that the patient --...,"[[4, 5, 0, 0, AGENT], [4, 5, 4, 5, DSE], [4, 5...",He|,his opinion|,the patient -- a woman|
3,The woman was admitted to the hospital on Satu...,"[[10, 10, 0, 1, AGENT], [10, 10, 10, 10, DSE],...",The woman|,complaining|,severe joint pains|
4,`` Since our technical equipment is far from p...,"[[22, 22, 2, 4, TARGET], [22, 22, 10, 10, TARG...",Nazarov|,said|,our technical equipment|we|
...,...,...,...,...,...
2444,Benjamin Franklin Federal Savings & Loan Assoc...,"[[9, 9, 8, 8, AGENT], [9, 9, 9, 9, DSE]]",it|,plans|,|
2445,thrift said the restructuring should help it m...,"[[1, 1, 1, 1, DSE], [1, 1, 2, 3, TARGET]]",|,said|,the restructuring|
2446,Details of the restructuring wo n't be made fi...,"[[11, 11, 10, 10, AGENT], [11, 11, 11, 11, DSE]]",regulators|,approve|,|
2447,"Jay Stevens , an analyst with Dean Witter Reyn...","[[12, 12, 11, 11, AGENT], [12, 12, 12, 12, DSE]]",he|,expected|,|


In [42]:
dev_df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.dev0.conll.json")

for i in range(len(dev_df)):
    agent, dse, target = organize_data(dev_df['orl'][i], dev_df['sep_sent'][i])
    dev_df.loc[i, 'agent'] = agent
    dev_df.loc[i, 'dse'] = dse
    dev_df.loc[i, 'target'] = target

dev_df.drop(columns='sep_sent')

,sentence,orl,agent,dse,target
0,The owner though that the animal was suffering...,"[[2, 2, 0, 1, AGENT], [2, 2, 2, 2, DSE], [2, 2...",The owner|,though|,the animal|
1,"The owner put down the animal , although the v...","[[10, 11, 8, 9, AGENT], [10, 11, 10, 11, DSE],...",the vet|,had forbidden|,him to do so|
2,"GATUNA , Rwanda , July 6 -LRB- AFP -RRB- - Pre...","[[30, 32, 30, 32, DSE], [30, 32, 33, 35, TARGET]]",|,soured relations between|,their neighbouring countries|
3,"The formerly close allies fell out in 1999 , t...","[[4, 5, 0, 3, AGENT], [4, 5, 4, 5, DSE], [4, 5...",The formerly close allies|The formerly close a...,fell out|mounting rivalry|,each| |
4,"In March , Uganda declared Rwanda a hostile na...","[[4, 4, 3, 3, AGENT], [4, 4, 4, 4, DSE], [4, 4...",Uganda| | |,declared|alleged|support|,Rwanda|Kigali|a rival to Museveni in a preside...
...,...,...,...,...,...
1033,He said the independent power segment could gr...,"[[1, 1, 0, 0, AGENT], [1, 1, 1, 1, DSE], [1, 1...",He|We|,said|expect|,the independent power segment|to supply a sign...
1034,Westinghouse also expects its international sa...,"[[2, 2, 0, 0, AGENT], [2, 2, 2, 2, DSE]]",Westinghouse|,expects|,|
1035,Analysts had expected Consolidated to post a s...,"[[2, 2, 0, 0, AGENT], [2, 2, 2, 2, DSE], [2, 2...",Analysts|,expected|,Consolidated to post a slim profit|
1036,"They have to continue to tighten their belts ,...","[[10, 10, 0, 0, TARGET], [10, 10, 10, 10, DSE]...","Craig Kloner , an analyst at Goldman , Sachs &...",said|,They|


In [43]:
test_df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.test0.conll.json")

for i in range(len(test_df)):
    agent, dse, target = organize_data(test_df['orl'][i], test_df['sep_sent'][i])
    test_df.loc[i, 'agent'] = agent
    test_df.loc[i, 'dse'] = dse
    test_df.loc[i, 'target'] = target

test_df.drop(columns='sep_sent')

,sentence,orl,agent,dse,target
0,Private organizations are also being encourage...,"[[2, 5, 2, 5, DSE], [2, 5, 7, 9, TARGET]]",|,are also being encouraged|,help fight sandstorms|
1,"It is believed that sand from northern areas ,...","[[1, 2, 1, 2, DSE], [1, 2, 4, 25, TARGET]]",|,is believed|,"sand from northern areas , especially the Inne..."
2,Russia favors creation of `` international ins...,"[[1, 1, 0, 0, AGENT], [1, 1, 1, 1, DSE], [1, 1...",Russia|,favors|,creation of `` international instruments '' to...
3,"Moscow , 16 July : The Russian Foreign Ministr...","[[9, 11, 5, 8, AGENT], [9, 11, 9, 11, DSE], [9...",The Russian Foreign Ministry|,has called for|,taking into account Russia 's specific interes...
4,A Foreign Ministry statement in connection wit...,"[[25, 25, 0, 3, AGENT], [25, 25, 25, 25, DSE],...",A Foreign Ministry statement|,reads|,taking into account Russia 's interests|
...,...,...,...,...,...
620,We will have a reasonably flat year this year ...,"[[13, 13, 4, 6, TARGET], [13, 13, 11, 12, AGEN...",Mr. Schwarz|,says|,reasonably flat year|
621,Mr. Schwarz also said costs associated with U....,"[[3, 3, 0, 1, AGENT], [3, 3, 3, 3, DSE], [3, 3...",Mr. Schwarz|,said|,costs associated with U.S. Trust 's planned move|
622,"Esselte AB , the Stockholm office supplies com...","[[12, 12, 0, 8, AGENT], [12, 12, 12, 12, DSE],...","Esselte AB , the Stockholm office supplies com...",proposed|,to acquire the 22 % it does n't own of its U.S...
623,A committee of outside directors for the Garde...,"[[20, 20, 18, 19, AGENT], [20, 20, 20, 20, DSE]]",the parent|,asked|,|


### Getting evaluation data into df and dividing each of {agent, target, dse} elements

In [44]:
def pipeDivider(pipedString):
    listOfItems = []
    listOfItems = pipedString.split('|')[:-1]
    return listOfItems

In [45]:
# function which will write prompt for the model according to the sentence and the items in it
def create_prompt(input_df):
    output_df = pd.DataFrame(columns=['target_prompt', 'agent_prompt', 'target', 'agent'])
    for i in range(len(input_df)):
        dse_list, target_list, agent_list = [], [], []
        dse_list = pipeDivider(str(input_df.iloc[i]['dse']))
        target_list = pipeDivider(str(input_df.iloc[i]['target']))
        agent_list = pipeDivider(str(input_df.iloc[i]['agent']))
        for j in range(len(dse_list)):
            last_row = int(len(output_df))+1
            output_df.loc[last_row, 'target_prompt'] = f"Sentence is: {input_df.iloc[i]['sentence']} Find target for this dse: {dse_list[j]}"
            output_df.loc[last_row, 'agent_prompt'] = f"Sentence is: {input_df.iloc[i]['sentence']} Find agent for this dse: {dse_list[j]}"
            output_df.loc[last_row, 'target'] = target_list[j]
            output_df.loc[last_row, 'agent'] = agent_list[j]
    return output_df

### Change the sentence and dse into a prompt according to information in that row 
#### change will be applied on all three data frames {train, test, validation}

In [46]:
dev_df = create_prompt(dev_df)
train_df = create_prompt(df)
test_df = create_prompt(test_df)

In [47]:
train_df = train_df.reset_index(drop=True)
dev_df = dev_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [48]:
from torch.utils.data import DataLoader, Dataset
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length4text, max_length4label):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length4text = max_length4text
        self.max_length4label = max_length4label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        agent_prompt = self.data['agent_prompt'][idx]
        target_prompt = self.data['target_prompt'][idx]
        agent = self.data['agent'][idx]
        target = self.data['target'][idx]
        # tokenizing agent prompt
        agent_prompt_encoding = self.tokenizer(agent_prompt, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        # tokenizing target prompt
        target_prompt_encoding = self.tokenizer(target_prompt, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        # tokenizing agent
        agent_encoding = self.tokenizer(agent, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        # tokenizing target
        target_encoding = self.tokenizer(target, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        return {
            # agent prompt
            'agent_input_id': agent_prompt_encoding['input_ids'].squeeze(),
            'agent_attention_mask': agent_prompt_encoding['attention_mask'].squeeze(),
            # target prompt
            'target_input_id': target_prompt_encoding['input_ids'].squeeze(),
            'target_attention_mask': target_prompt_encoding['attention_mask'].squeeze(),
            #agent
            'agent_id': agent_encoding['input_ids'].squeeze(),
            'agent_mask': agent_encoding['attention_mask'].squeeze(),
            #target
            'target_id': target_encoding['input_ids'].squeeze(),
            'target_mask': target_encoding['attention_mask'].squeeze()
        }

In [49]:
# Create a data loader for TRAIN dataframe 
train_dataset = CustomDataset(train_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
train_data_loader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle=True)

# Create a data loader for EVALUATION dataframe
val_dataset = CustomDataset(dev_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
val_data_loader = DataLoader(val_dataset, batch_size= BATCH_SIZE, shuffle=False)

# Create a data loader for TEST dataframe
test_dataset = CustomDataset(test_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
test_data_loader = DataLoader(test_dataset, batch_size= BATCH_SIZE, shuffle=False)

# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [50]:
def f1_calculator(pred_list, actual_list):
    cleaned_pred, cleaned_actual = [], []
    matched = 0
    for i in range(len(pred_list)):
        for j in range(len(actual_list)):
            if actual_list[j] in pred_list:
                matched += 1
        cleaned_pred.extend(pred_list)
        cleaned_actual.extend(actual_list)
    
    prediction_len = len(cleaned_pred)
    actual_len = len(cleaned_actual)
    print(f'matched: {matched}, prediction_len:{prediction_len}, actual_len:{actual_len} \n')
    try:
        precision = (matched / prediction_len)
        recall = (matched / actual_len)
        f1 = (2 * (precision * recall)) / (precision + recall)
    except:
        f1 = 0
    return f1

In [51]:
def evaluate_model(model, dataloader, prompt_type):
    if prompt_type == 'target':
        id_type = 'target_input_id'
        attention_type = 'target_attention_mask'
        output_type = 'target_id'
    elif prompt_type == 'agent':
        id_type = 'agent_input_id'
        attention_type = 'agent_attention_mask'
        output_type = 'agent_id'
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        true_counted, prediction_counted, actual_counted = 0,0,0
        matched, prediction_len, actual_len = 0,0,0
        actual_list, prediction_list, f1 = [], [], []
        for batch_idx, batch in enumerate(dataloader):
            # Move data to the specified device
            # batch = {key: value.to('cuda') for key, value in batch.items()}

            # Forward pass
            ids = batch[id_type]
            mask = batch[attention_type]
            output_id = batch[output_type]
            
            actuals = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in output_id]
            
            generated_output = generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=64, 
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_output]
            
            actual_list.extend(actuals)
            prediction_list.extend(preds)
    return f1_calculator(prediction_list, actual_list)

In [52]:
model.train()
for epoch in range(EPOCH):
    losses = []
    for batch in train_data_loader:
        
        agent_input = batch['agent_input_id']
        agent_attention_mask = batch['agent_attention_mask']
        
        target_input = batch['target_input_id']
        target_attention_mask = batch['target_attention_mask']
        
        agent_id = batch['agent_id']
        agent_mask = batch['agent_mask']
        
        target_id = batch['target_id']
        target_mask = batch['target_mask']
        
        optimizer.zero_grad()
        
        agent_output = model(agent_input, attention_mask=agent_attention_mask, labels=agent_id)
        target_output = model(target_input, attention_mask=target_attention_mask, labels=target_id)
        
        agent_loss = agent_output.loss
        target_loss = target_output.loss
        losses.append(agent_loss.item())
        losses.append(target_loss.item())
        
        agent_loss.backward()
        target_loss.backward()
        optimizer.step()
        
    f1_4_target = evaluate_model(model, val_data_loader, 'target')
    f1_4_agent = evaluate_model(model, val_data_loader, 'agent')
    print(f'loss: {np.mean(losses)}, f1 validation for target:{f1_4_target}, f1 validation for agent:{f1_4_agent} \n end of epoch{epoch}. \n')

matched: 1278123, prediction_len:2277081, actual_len:2277081 

matched: 1815327, prediction_len:2277081, actual_len:2277081 

loss: 1.3896529511358944, f1 validation for target:0.56129887342611, f1 validation for agent:0.7972166998011928 
 end of epoch0. 

matched: 1341501, prediction_len:2277081, actual_len:2277081 

matched: 1771566, prediction_len:2277081, actual_len:2277081 

loss: 0.2913709651189962, f1 validation for target:0.5891318754141815, f1 validation for agent:0.777998674618953 
 end of epoch1. 

matched: 1284159, prediction_len:2277081, actual_len:2277081 

matched: 1761003, prediction_len:2277081, actual_len:2277081 

loss: 0.19329964189592125, f1 validation for target:0.563949635520212, f1 validation for agent:0.7733598409542743 
 end of epoch2. 

matched: 1365645, prediction_len:2277081, actual_len:2277081 

matched: 1794201, prediction_len:2277081, actual_len:2277081 

loss: 0.14116531449509154, f1 validation for target:0.5997349237905898, f1 validation for agent:0.78

### Here we can either use the model we trained above, or we can use the link to the same model.

In [53]:
# defining a new model to fit the state dictionary of the previously trained model to it.
new_model = T5ForConditionalGeneration.from_pretrained(MODEL_LINK).to('cuda')

state_dict = torch.load("/kaggle/input/t5-small_trained_mpqa/pytorch/t5-small_mpqa_v1/1/t5_orl_model.pt")
new_model.load_state_dict(state_dict)

<All keys matched successfully>

### Here we are gonna load the test dataset into evaluation method to calculate score

In [54]:
target_accuracy = evaluate_model(new_model, test_data_loader, 'target')
agent_accuracy = evaluate_model(new_model, test_data_loader, 'agent')
print(f'Agent accuracy: {agent_accuracy}. Target accuracy: {target_accuracy}\n\n----------------------')

matched: 475076, prediction_len:797449, actual_len:797449 

matched: 642067, prediction_len:797449, actual_len:797449 

Agent accuracy: 0.8051511758118701. Target accuracy: 0.5957446808510638

----------------------
